In [5]:
!pip install azure-cognitiveservices-vision-customvision


     |████████████████████████████████| 112kB 1.5MB/s eta 0:00:01    |███████████████                 | 51kB 1.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region

ENDPOINT = "https://westeurope.api.cognitive.microsoft.com/"

# Replace with a valid key
training_key = "ee96ce4c988e46618994b8fba546326f"
prediction_key = "ee96ce4c988e46618994b8fba546326f"
prediction_resource_id = "/subscriptions/d0c96ed6-4a68-4c71-96d1-08b497bca696/resourceGroups/customvision/providers/Microsoft.CognitiveServices/accounts/customvision"

publish_iteration_name = "detectModel"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [11]:
raim_tag = trainer.create_tag(project.id, "raim")

In [15]:
import os
import pandas as pd
import numpy as np

base_image_url = "./images"


# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []

for root, folders, filenames in os.walk(base_image_url):
    for file_name in filenames:
        if file_name.endswith(".png") or file_name.endswith(".jpg"):
            print (file_name)
            regions = []

            df=pd.read_csv(base_image_url + "/" + file_name[:-4] + ".txt",header=None, sep=' ')
            df.columns=['c', 'x','y', 'w', 'h']

            #print (df.values)

            regions = []
            for i in df.values:
                 regions += [ Region(tag_id=raim_tag.id, left=i[1]-i[3]/2,top=i[2]-i[4]/2,width=i[3],height=i[4]) ]

            #print(regions)

            with open(base_image_url + "/" + file_name, mode="rb") as image_contents:
                tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    

Adding images...
CDY_2016.jpg
CDY_2015.jpg
